In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

2023-07-20 21:14:44.727755: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# read the file that has smiles and ids

In [3]:
pubchem_to_drugs_df = pd.read_csv('../data/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv')

In [4]:
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]

In [5]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [6]:
pubchem_to_drugs_df["PubCHEM"] = [val if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]

In [7]:
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()

In [8]:
pubchem_to_drugs_df["PubCHEM"] = pubchem_to_drugs_df["PubCHEM"].astype(np.int64)

In [9]:
pubchem_to_smiles = pd.read_csv('../data/223drugs_pubchem_smiles.txt', sep="\t", header=None)

In [10]:
pubchem_to_smiles.columns = ["PubCHEM", "Smiles"]

In [11]:
pubchem_to_smiles["PubCHEM"] = pubchem_to_smiles["PubCHEM"].astype(np.int64)

In [12]:
pubchem_drugs_smiles_df = pubchem_to_drugs_df.merge(pubchem_to_smiles, on = "PubCHEM")

In [13]:
def get_emb_models(dataset, id_col):
    # std = StandardScaler()
    unique_ids = dataset[id_col].values
    text_vec_layer = tf.keras.layers.TextVectorization(max_tokens = dataset.shape[0] + 2, 
                                                  standardize=None, split = None, 
                                                  output_mode = "int", 
                                                  vocabulary = unique_ids.tolist())
    weights = dataset.drop(id_col, 1).values
    padding_zeros = np.zeros((2, weights.shape[1]))
    weights = np.vstack((padding_zeros, weights))
    # std.fit(weights)
    # weights = std.transform(weights)
    emb_layer = tf.keras.layers.Embedding(dataset.shape[0] + 2, 
                                     weights.shape[1], 
                                     weights = [weights], 
                                     trainable = False)
    input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    vec_out = text_vec_layer(input_layer)
    emb_out = emb_layer(vec_out)
    flat_out = tf.keras.layers.Flatten()(emb_out)
    emb_model = tf.keras.models.Model(input_layer, flat_out)
    return emb_model

In [14]:
selected_info_common_cell_lines = "../data/CCLE/cellline_list.txt"
selected_info_common_genes = "../data/CCLE/gene_list.txt"

In [15]:
with open(selected_info_common_genes) as f:
    common_genes = [item.strip() for item in f.readlines()]

In [16]:
len(common_genes)

697

In [18]:
# read cancer cell line data
cancer_cell_copy_num_df = pd.read_csv('../data/CCLE/genomic_copynumber_561celllines_710genes_demap_features.csv')

In [19]:
cancer_cell_copy_num_df.shape

(561, 711)

In [20]:
cancer_cell_copy_num_df.head()

,Unnamed: 0,AKT3,ABI1,SH2B3,CDH10,CDH11,AKAP9,CDH17,LHFP,CDK4,...,CD79B,BCLAF1,KEAP1,SETDB1,SRGAP3,MAFB,GOLGA5,THRAP3,MED12,CDH1
0,ACH-000828,1.548332,1.040041,1.020747,1.374471,1.042200,0.848321,1.729350,0.847032,1.025471,...,1.121307,0.830675,0.879632,1.492768,1.053714,1.014371,1.213421,0.780331,1.013260,0.641870
1,ACH-000568,0.798791,1.072012,1.038193,1.061508,0.792855,1.061508,1.271905,0.779829,1.002602,...,1.316797,1.049884,0.821346,1.621861,1.291863,1.046269,0.799812,1.074006,1.005510,0.792855
2,ACH-000560,1.080266,0.864616,0.694564,1.053766,0.628903,1.423261,1.369500,0.632267,0.911728,...,1.100098,1.069393,1.232225,0.860476,1.063332,1.194632,1.050246,0.894051,0.873167,0.620049
3,ACH-000561,1.032649,1.408964,0.949651,1.293622,1.274622,1.503638,1.470401,1.198447,0.893277,...,1.063162,0.660578,0.892464,1.252069,0.650263,1.178741,1.052244,0.902153,0.681885,1.274622
4,ACH-000562,1.406262,1.005521,0.997044,1.015986,0.932915,1.007777,1.051055,0.822244,0.997044,...,1.192299,0.974316,0.997944,1.406262,0.810652,1.008153,0.943241,0.878888,0.608959,0.932915


In [28]:
cancer_cell_copy_num_df[common_genes]

KeyError: "['ZMYM2', 'RANBP17', 'PCSK7', 'TTL', 'CHN1', 'PATZ1'] not in index"

In [21]:
cancer_cell_copy_num_df = cancer_cell_copy_num_df.fillna(cancer_cell_copy_num_df.mean())

/tmp/ipykernel_1382693/1778569368.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cancer_cell_copy_num_df = cancer_cell_copy_num_df.fillna(cancer_cell_copy_num_df.mean())


In [22]:
int1 = cancer_cell_copy_num_df.columns[1:].tolist()

In [23]:
# read cancer cell line data
cancer_cell_gen_expr_df = pd.read_csv('../data/CCLE/genomic_expression_561celllines_697genes_demap_features.csv')

In [24]:
cancer_cell_gen_expr_df = cancer_cell_gen_expr_df.fillna(cancer_cell_gen_expr_df.mean())

/tmp/ipykernel_1382693/1063970889.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cancer_cell_gen_expr_df = cancer_cell_gen_expr_df.fillna(cancer_cell_gen_expr_df.mean())


In [29]:
cancer_cell_gen_expr_df[common_genes]

KeyError: "['ZMYM2', 'RANBP17', 'PCSK7', 'TTL', 'CHN1', 'PATZ1'] not in index"

In [25]:
int2 = cancer_cell_gen_expr_df.columns[1:].tolist()

In [27]:
len(set(int2).intersection(common_genes))

691

In [24]:
common_genes = set(common_genes).intersection(int1)

In [25]:
common_genes = set(common_genes).intersection(int2)

In [26]:
# len(common_genes)

In [27]:
cancer_cell_copy_num_df = pd.concat([cancer_cell_copy_num_df.iloc[:,0], cancer_cell_copy_num_df.iloc[:,1:][common_genes]], axis = 1)

/tmp/ipykernel_579255/2924742545.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  cancer_cell_copy_num_df = pd.concat([cancer_cell_copy_num_df.iloc[:,0], cancer_cell_copy_num_df.iloc[:,1:][common_genes]], axis = 1)


In [28]:
# common_genes

In [29]:
common_genes = list(common_genes)

In [30]:
import pickle

In [31]:
with open("..//data//common_genes.pickle", "wb") as f:
    pickle.dump(common_genes, f)

In [32]:
cancer_cell_gen_expr_df = pd.concat([cancer_cell_gen_expr_df.iloc[:,0], cancer_cell_gen_expr_df.iloc[:,1:][common_genes]], axis = 1)

In [33]:
cancer_cell_gen_expr_df

,Unnamed: 0,PTCH1,LATS1,ERBB2,BCL10,NCOR2,NRAS,CCNE1,ACKR3,PRDM2,...,BRCA2,POLG,PHOX2B,GOPC,PRPF40B,CIC,HOXC11,EIF1AX,USP8,ALK
0,ACH-000828,0.695994,2.713696,11.456611,3.349082,6.412104,3.106013,3.723559,1.974529,3.499527,...,2.211012,5.301222,0.000000,3.612352,4.625855,4.649041,4.373648,5.694602,4.767125,0.000000
1,ACH-000568,2.169925,2.776104,10.959857,2.049631,5.475085,1.981853,3.661065,2.490570,2.432959,...,2.056584,4.800641,0.000000,3.764474,5.007644,4.688180,2.568032,5.239169,3.482848,0.070389
2,ACH-000560,3.884598,3.849999,3.300124,1.799087,5.960929,4.488001,4.676380,0.594549,3.799087,...,2.516015,5.763412,0.014355,5.245267,3.960697,5.780048,1.914565,6.024586,4.878725,0.070389
3,ACH-000561,4.297925,2.877744,5.804776,2.653060,5.077243,7.098769,2.952334,6.689858,4.009885,...,2.430285,6.017031,0.000000,3.598127,3.957915,3.948601,1.485427,6.373474,5.316870,0.422233
4,ACH-000562,1.201634,2.653060,5.973611,4.746850,5.421223,3.988230,4.313246,0.238787,3.147307,...,2.599318,5.459759,0.014355,3.761285,3.658783,4.100978,0.000000,5.681449,4.184280,0.275007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,ACH-000242,1.485427,3.035624,6.441616,3.588565,5.997292,4.781360,4.280956,7.292506,3.324811,...,3.401903,5.643568,0.000000,3.375735,3.021480,5.087463,0.773996,6.121844,5.134632,0.014355
557,ACH-000245,3.135863,2.906891,1.516015,2.669027,5.025472,4.970394,4.829850,0.695994,5.239551,...,4.418865,6.417684,0.014355,4.631686,1.989139,4.620586,0.298658,6.216649,4.616475,0.000000
558,ACH-000244,1.469886,3.111031,5.431623,4.002703,6.752882,5.249825,3.326250,0.238787,3.600508,...,1.214125,5.351204,0.000000,4.429616,3.724650,4.991408,0.879706,6.822475,5.260402,0.263034
559,ACH-000247,3.134221,2.803227,6.014802,3.019702,6.217812,4.491212,3.976364,0.056584,3.566815,...,4.054848,5.666188,0.000000,4.118526,2.799087,4.274262,3.529821,6.670585,4.530445,0.000000


In [34]:
cancer_copy_number_model = get_emb_models(cancer_cell_copy_num_df, "Unnamed: 0")

2023-07-18 17:16:13.066698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2023-07-18 17:16:13.066738: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-18 17:16:13.066765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c2518.swan.hcc.unl.edu): /proc/driver/nvidia/version does not exist
2023-07-18 17:16:13.067129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_579255/367420769.py:8: Futu

In [35]:
cancer_copy_number_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 1)                0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 1, 691)            389033    
                                                                 
 flatten (Flatten)           (None, 691)               0         
                                                                 
Total params: 389,033
Trainable params: 0
Non-trainable params: 389,033
_________________________________________________________________


In [36]:
cancer_copy_number_model.save("..//Models//cancer_copy_number_model_no_norm_common")

INFO:tensorflow:Assets written to: ..//Models//cancer_copy_number_model_no_norm_common/assets


INFO:tensorflow:Assets written to: ..//Models//cancer_copy_number_model_no_norm_common/assets


In [37]:
cancer_cell_gen_expr_df.shape

(561, 692)

In [38]:
cancer_cell_gen_expr_df.head()

,Unnamed: 0,PTCH1,LATS1,ERBB2,BCL10,NCOR2,NRAS,CCNE1,ACKR3,PRDM2,...,BRCA2,POLG,PHOX2B,GOPC,PRPF40B,CIC,HOXC11,EIF1AX,USP8,ALK
0,ACH-000828,0.695994,2.713696,11.456611,3.349082,6.412104,3.106013,3.723559,1.974529,3.499527,...,2.211012,5.301222,0.000000,3.612352,4.625855,4.649041,4.373648,5.694602,4.767125,0.000000
1,ACH-000568,2.169925,2.776104,10.959857,2.049631,5.475085,1.981853,3.661065,2.490570,2.432959,...,2.056584,4.800641,0.000000,3.764474,5.007644,4.688180,2.568032,5.239169,3.482848,0.070389
2,ACH-000560,3.884598,3.849999,3.300124,1.799087,5.960929,4.488001,4.676380,0.594549,3.799087,...,2.516015,5.763412,0.014355,5.245267,3.960697,5.780048,1.914565,6.024586,4.878725,0.070389
3,ACH-000561,4.297925,2.877744,5.804776,2.653060,5.077243,7.098769,2.952334,6.689858,4.009885,...,2.430285,6.017031,0.000000,3.598127,3.957915,3.948601,1.485427,6.373474,5.316870,0.422233
4,ACH-000562,1.201634,2.653060,5.973611,4.746850,5.421223,3.988230,4.313246,0.238787,3.147307,...,2.599318,5.459759,0.014355,3.761285,3.658783,4.100978,0.000000,5.681449,4.184280,0.275007


In [39]:
cancer_cell_gen_expr_model = get_emb_models(cancer_cell_gen_expr_df, "Unnamed: 0")

/tmp/ipykernel_579255/367420769.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  weights = dataset.drop(id_col, 1).values


In [40]:
cancer_cell_gen_expr_model.save("..//Models//cancer_cell_gen_expr_model_no_norm_common")

INFO:tensorflow:Assets written to: ..//Models//cancer_cell_gen_expr_model_no_norm_common/assets


INFO:tensorflow:Assets written to: ..//Models//cancer_cell_gen_expr_model_no_norm_common/assets


In [41]:
cancer_cell_gen_expr_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 1)                0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 1, 691)            389033    
                                                                 
 flatten_1 (Flatten)         (None, 691)               0         
                                                                 
Total params: 389,033
Trainable params: 0
Non-trainable params: 389,033
_________________________________________________________________
